# **Twitter API:**

Using tweepy, which is an easy-to-use Python library for accessing the Twitter API



### **Setting up the environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Installing requirements

In [2]:
%%capture
!pip install tweepy

###**Importing the required modules**

In [3]:
# import relevant libraries
import tweepy
import pandas as pd
import numpy as np
import os
import csv

### **Connecting to twitter**

In [4]:
# load twitter API secrets from an external file
config = pd.read_csv("/content/drive/MyDrive/Twitter Analysis/config.csv")

twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [5]:
# function to get all tweets for a specified keyword
def get_all_tweets(keyword, count):
  '''
  input:
    keyword: a chosen hashtag
    count: the number of tweets to extract
  output:
    csv: a csv file containing time and text
  '''  
  # authorize access
  auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
  auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
  api = tweepy.API(auth, wait_on_rate_limit=True)

  # initialize list and fetch tweets
  alltweets = []
  new_tweets = tweepy.Cursor(api.search, q=keyword, tweet_mode="extended").items(count)
  alltweets.extend(new_tweets)

  # transform the tweepy tweets into a 2D array that will populate the csv
  outtweets = [[tweet.id_str, tweet.created_at, tweet.full_text] for tweet in alltweets]

  # write to csv
  with open(f'/content/drive/MyDrive/Twitter Analysis/data/new_{keyword}_tweets.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["id", "created_at", "text"])
    display(outtweets)
    writer.writerows(outtweets)
  pass

In [ ]:
# call the function to get the most recent 6000 tweets containing the hashtag #soeder
%%capture
get_all_tweets('soeder', 6000)

In [ ]:
# call the function to get the most recent 6000 tweets containing the hashtag #laschet
%%capture
get_all_tweets('laschet', 6000)